In [2]:
import lux
import os
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import strip_accents_unicode

DATA_PROJECTS_FILE_CSV = "data/projects.csv"
DATA_PROJECTS_FILE_H5 = "data/projects.h5"
DATA_PROJECTS_URL = 'https://filesender.renater.fr/?s=download&token=2f5ed948-6e35-4bf1-88ed-7ab5dd0411b9'

def load(path, cache=False):
    # Reload previously processed dataset, if available
    if (os.path.isfile(path)):
        print('> Load compressed HD5 file...')
        df = pd.read_hdf(path, 'data', mode='r')
        return df

    # Download CSV file, if necessary
    # if not os.path.isfile(DATA_PROJECTS_FILE_CSV):
    #     self.__download_data(DATA_PROJECTS_FILE_CSV)

    # Read original CSV file
    print("> Loading from raw CSV file...")
    df = pd.read_csv(DATA_PROJECTS_FILE_CSV, encoding='Windows-1252', na_values=[None], parse_dates=['start_date','end_date'], date_parser=self.dateparse)

    # Preprocess original dataset
    print("> Preprocessing...")
    print('> Convert textual columns to \'str\'...')
    labels = [
        'id',
    ]
    for label in labels:
        df[label] = df.loc[df[label].notna(), label].astype(str)

    print('> Normalizing string columns...')
    for label, content in df.select_dtypes(object).iteritems():
        df[label] = df[label].apply(lambda x: strip_accents_unicode(str(x).upper().strip()) if not str(x) in (None, 'nan') else None)
        df.loc[(df[label] == ''), label] = None
        df[label].replace('\s+', ' ', regex=True, inplace=True)
    
    # Save the DataFrame in HDF5 for faster reload.
    print('> Save compressed and processed DataFrame...')
    df.to_hdf(path, key='data', complevel=9)

    return df

df:DataFrame = load(DATA_PROJECTS_FILE_H5)


> Load compressed HD5 file...


In [3]:
df = df.drop(['id', 'name', 'start_date', 'end_date'], axis=1)
# df.info()
# df = pd.read_csv("https://github.com/lux-org/lux-datasets/blob/master/data/hpi_cleaned.csv?raw=True")
# df.info()
# df = pd.read_csv("https://raw.githubusercontent.com/lux-org/lux-datasets/master/data/college.csv")
df